In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import shutil
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm_notebook as tqdm

from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt

from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel, ExponentialSmoothing, BlockRNNModel
from darts.metrics import mape
from darts.utils.statistics import check_seasonality, plot_acf
from darts.datasets import AirPassengersDataset, SunspotsDataset
from darts.utils.timeseries_generation import datetime_attribute_timeseries

import warnings

warnings.filterwarnings("ignore")
import logging

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import BatchNormalization

from datetime import datetime

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from numpy import asarray
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from matplotlib import pyplot

2022-09-09 09:46:19.526361: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-09 09:46:19.645894: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-09-09 09:46:19.650053: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-09 09:46:19.650063: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [3]:
df_rpi = pd.read_csv('RPI_CPI.csv')
# df_rpi['Date'] = pd.to_datetime(df_rpi['Date'], infer_datetime_format=True)
# df_rpi = df_rpi.set_index('Date')
df_rpi

,Date,RPI,CPI
0,1987-01-01,100.0,0.0
1,1987-02-01,100.4,0.0
2,1987-03-01,100.6,0.0
3,1987-04-01,101.8,0.0
4,1987-05-01,101.9,0.0
...,...,...,...
422,2022-03-01,323.5,117.1
423,2022-04-01,334.6,120.0
424,2022-05-01,337.1,120.8
425,2022-06-01,340.0,121.8


In [4]:
df = pd.read_csv('BloombergData-01a.csv')
df = df.drop(columns= ['Date BPSW2 Curncy', 'BPSW2 Curncy','Date BPSW10 Curncy','BPSW10 Curncy'] )


In [5]:
df

,Date CO1 Comdty,CO1 Comdty,Date CL1 Comdty,CL1 Comdty,Date GBP Curncy,GBP Curncy,Date GBPEUR Curncy,GBPEUR Curncy,Date UKX Index,UKX Index,Date SPX Index,SPX Index,Date SX5E Index,SX5E Index,Date TTFG1MON Index,TTFG1MON Index
0,23/06/1988,15.65,02/01/1987,18.13,01/01/1987,1.4825,01/01/1999,1.4079,01/01/1987,1679.0,02/01/1987,246.45,01/01/1987,900.82,03/12/2003,14.65
1,24/06/1988,15.65,05/01/1987,17.95,02/01/1987,1.4900,04/01/1999,1.4020,02/01/1987,1681.1,05/01/1987,252.19,02/01/1987,891.78,04/12/2003,14.75
2,27/06/1988,15.10,06/01/1987,18.22,05/01/1987,1.4728,05/01/1999,1.4012,05/01/1987,1680.0,06/01/1987,252.78,05/01/1987,898.33,05/12/2003,14.75
3,28/06/1988,15.27,07/01/1987,18.25,06/01/1987,1.4762,06/01/1999,1.4236,06/01/1987,1690.7,07/01/1987,255.33,06/01/1987,902.32,08/12/2003,14.63
4,29/06/1988,14.97,08/01/1987,18.57,07/01/1987,1.4705,07/01/1999,1.4130,07/01/1987,1722.2,08/01/1987,257.29,07/01/1987,899.15,09/12/2003,14.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9303,NaN,NaN,NaN,NaN,30/08/2022,1.1656,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9304,NaN,NaN,NaN,NaN,31/08/2022,1.1622,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9305,NaN,NaN,NaN,NaN,01/09/2022,1.1545,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9306,NaN,NaN,NaN,NaN,02/09/2022,1.1509,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df = df.join(df_rpi)
df

,Date CO1 Comdty,CO1 Comdty,Date CL1 Comdty,CL1 Comdty,Date GBP Curncy,GBP Curncy,Date GBPEUR Curncy,GBPEUR Curncy,Date UKX Index,UKX Index,Date SPX Index,SPX Index,Date SX5E Index,SX5E Index,Date TTFG1MON Index,TTFG1MON Index,Date,RPI,CPI
0,23/06/1988,15.65,02/01/1987,18.13,01/01/1987,1.4825,01/01/1999,1.4079,01/01/1987,1679.0,02/01/1987,246.45,01/01/1987,900.82,03/12/2003,14.65,1987-01-01,100.0,0.0
1,24/06/1988,15.65,05/01/1987,17.95,02/01/1987,1.4900,04/01/1999,1.4020,02/01/1987,1681.1,05/01/1987,252.19,02/01/1987,891.78,04/12/2003,14.75,1987-02-01,100.4,0.0
2,27/06/1988,15.10,06/01/1987,18.22,05/01/1987,1.4728,05/01/1999,1.4012,05/01/1987,1680.0,06/01/1987,252.78,05/01/1987,898.33,05/12/2003,14.75,1987-03-01,100.6,0.0
3,28/06/1988,15.27,07/01/1987,18.25,06/01/1987,1.4762,06/01/1999,1.4236,06/01/1987,1690.7,07/01/1987,255.33,06/01/1987,902.32,08/12/2003,14.63,1987-04-01,101.8,0.0
4,29/06/1988,14.97,08/01/1987,18.57,07/01/1987,1.4705,07/01/1999,1.4130,07/01/1987,1722.2,08/01/1987,257.29,07/01/1987,899.15,09/12/2003,14.63,1987-05-01,101.9,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9303,NaN,NaN,NaN,NaN,30/08/2022,1.1656,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9304,NaN,NaN,NaN,NaN,31/08/2022,1.1622,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9305,NaN,NaN,NaN,NaN,01/09/2022,1.1545,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9306,NaN,NaN,NaN,NaN,02/09/2022,1.1509,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
swaps = pd.read_csv('GBP_IRS_2y_AND_10y.csv')
swaps

,Date_GBP_IRS_2y,GBP_IRS_2y,Date_GBP_IRS_10y,GBP_IRS_10y
0,30/09/2009,1.8224,01/10/2009,3.79000
1,01/10/2009,1.7500,02/10/2009,3.75000
2,02/10/2009,1.7375,05/10/2009,3.74500
3,05/10/2009,1.7275,06/10/2009,3.75000
4,06/10/2009,1.7550,07/10/2009,3.73500
...,...,...,...,...
3130,NaN,NaN,23/12/2021,1.18110
3131,NaN,NaN,24/12/2021,1.17500
3132,NaN,NaN,29/12/2021,1.26775
3133,NaN,NaN,30/12/2021,1.22720


In [8]:
df = df.join(swaps)

In [9]:
df

,Date CO1 Comdty,CO1 Comdty,Date CL1 Comdty,CL1 Comdty,Date GBP Curncy,GBP Curncy,Date GBPEUR Curncy,GBPEUR Curncy,Date UKX Index,UKX Index,...,SX5E Index,Date TTFG1MON Index,TTFG1MON Index,Date,RPI,CPI,Date_GBP_IRS_2y,GBP_IRS_2y,Date_GBP_IRS_10y,GBP_IRS_10y
0,23/06/1988,15.65,02/01/1987,18.13,01/01/1987,1.4825,01/01/1999,1.4079,01/01/1987,1679.0,...,900.82,03/12/2003,14.65,1987-01-01,100.0,0.0,30/09/2009,1.8224,01/10/2009,3.790
1,24/06/1988,15.65,05/01/1987,17.95,02/01/1987,1.4900,04/01/1999,1.4020,02/01/1987,1681.1,...,891.78,04/12/2003,14.75,1987-02-01,100.4,0.0,01/10/2009,1.7500,02/10/2009,3.750
2,27/06/1988,15.10,06/01/1987,18.22,05/01/1987,1.4728,05/01/1999,1.4012,05/01/1987,1680.0,...,898.33,05/12/2003,14.75,1987-03-01,100.6,0.0,02/10/2009,1.7375,05/10/2009,3.745
3,28/06/1988,15.27,07/01/1987,18.25,06/01/1987,1.4762,06/01/1999,1.4236,06/01/1987,1690.7,...,902.32,08/12/2003,14.63,1987-04-01,101.8,0.0,05/10/2009,1.7275,06/10/2009,3.750
4,29/06/1988,14.97,08/01/1987,18.57,07/01/1987,1.4705,07/01/1999,1.4130,07/01/1987,1722.2,...,899.15,09/12/2003,14.63,1987-05-01,101.9,0.0,06/10/2009,1.7550,07/10/2009,3.735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9303,NaN,NaN,NaN,NaN,30/08/2022,1.1656,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9304,NaN,NaN,NaN,NaN,31/08/2022,1.1622,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9305,NaN,NaN,NaN,NaN,01/09/2022,1.1545,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9306,NaN,NaN,NaN,NaN,02/09/2022,1.1509,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
bp_df= pd.read_csv('BPSW2_and_10.csv')
bp_df

,Date BPSW2 TRPU Curncy,BPSW2 TRPU Curncy,Date BPSW10 TRPU Curncy,BPSW10 TRPU Curncy
0,10/07/2006,5.0500,10/07/2006,5.0125
1,11/07/2006,5.0400,11/07/2006,4.9925
2,12/07/2006,5.0625,12/07/2006,5.0200
3,13/07/2006,5.0375,13/07/2006,5.0050
4,14/07/2006,5.0000,14/07/2006,4.9375
...,...,...,...,...
3091,30/08/2022,4.0901,NaN,NaN
3092,31/08/2022,4.1275,NaN,NaN
3093,01/09/2022,4.1860,NaN,NaN
3094,02/09/2022,4.1411,NaN,NaN


In [11]:
columns_list = list(bp_df.columns)
for columns in columns_list:
    if 'Date' in columns:
        bp_df[columns] = pd.to_datetime(bp_df[columns], infer_datetime_format=True)
bp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3096 entries, 0 to 3095
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Date BPSW2 TRPU Curncy   3096 non-null   datetime64[ns]
 1   BPSW2 TRPU Curncy        3096 non-null   float64       
 2   Date BPSW10 TRPU Curncy  3080 non-null   datetime64[ns]
 3   BPSW10 TRPU Curncy       3080 non-null   float64       
dtypes: datetime64[ns](2), float64(2)
memory usage: 96.9 KB


In [12]:
columns_list = list(df.columns)
for columns in columns_list:
    if 'Date' in columns:
        df[columns] = pd.to_datetime(df[columns], infer_datetime_format=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9308 entries, 0 to 9307
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Date CO1 Comdty      8735 non-null   datetime64[ns]
 1   CO1 Comdty           8735 non-null   float64       
 2   Date CL1 Comdty      8969 non-null   datetime64[ns]
 3   CL1 Comdty           8969 non-null   float64       
 4   Date GBP Curncy      9308 non-null   datetime64[ns]
 5   GBP Curncy           9308 non-null   float64       
 6   Date GBPEUR Curncy   6176 non-null   datetime64[ns]
 7   GBPEUR Curncy        6176 non-null   float64       
 8   Date UKX Index       9021 non-null   datetime64[ns]
 9   UKX Index            9021 non-null   float64       
 10  Date SPX Index       8991 non-null   datetime64[ns]
 11  SPX Index            8991 non-null   float64       
 12  Date SX5E Index      9195 non-null   datetime64[ns]
 13  SX5E Index           9195 non-nul

In [13]:
BPSW2_Curncy = bp_df.groupby(pd.PeriodIndex(bp_df['Date BPSW2 TRPU Curncy'], freq='M'))[['BPSW2 TRPU Curncy']].mean()
BPSW10_Curncy = bp_df.groupby(pd.PeriodIndex(bp_df['Date BPSW10 TRPU Curncy'], freq='M'))[['BPSW10 TRPU Curncy']].mean()

In [14]:
bp_df_list = [BPSW2_Curncy,BPSW10_Curncy]


In [15]:
bp_result_df = bp_df_list[0]
for i in range(len(bp_df_list)-1):
    if i<len(bp_df_list):
        bp_result_df = bp_result_df.join(bp_df_list[i+1], how='left')

In [16]:
bp_result_df

,BPSW2 TRPU Curncy,BPSW10 TRPU Curncy
Date BPSW2 TRPU Curncy,,
2006-01,5.196875,4.910000
2006-02,5.216667,4.965833
2006-03,5.276667,5.019167
2006-04,5.220625,4.951250
2006-05,5.217500,4.913333
...,...,...
2022-08,3.368889,2.688500
2022-09,2.475340,2.155940
2022-10,2.349517,2.063400


In [17]:
bp_result_df.index = bp_result_df.index.to_timestamp()

In [18]:
CO1_Comdty = df.groupby(pd.PeriodIndex(df['Date CO1 Comdty'], freq='M'))[['CO1 Comdty']].mean()
CL1_Comdty = df.groupby(pd.PeriodIndex(df['Date CL1 Comdty'], freq='M'))[['CL1 Comdty']].mean()
GBP_Curncy = df.groupby(pd.PeriodIndex(df['Date GBP Curncy'], freq='M'))[['GBP Curncy']].mean()
GBPEUR_Curncy = df.groupby(pd.PeriodIndex(df['Date GBPEUR Curncy'], freq='M'))[['GBPEUR Curncy']].mean()
UKX_Index = df.groupby(pd.PeriodIndex(df['Date UKX Index'], freq='M'))[['UKX Index']].mean()
SPX_Index = df.groupby(pd.PeriodIndex(df['Date SPX Index'], freq='M'))[['SPX Index']].mean()
SX5E_Index = df.groupby(pd.PeriodIndex(df['Date SX5E Index'], freq='M'))[['SX5E Index']].mean()
# BPSW2_Curncy = df.groupby(pd.PeriodIndex(df['Date BPSW2 TRPU Curncy'], freq='M'))[['BPSW2 TRPU Curncy']].mean()
# BPSW10_Curncy = df.groupby(pd.PeriodIndex(df['Date BPSW10 TRPU Curncy'], freq='M'))[['BPSW10 TRPU Curncy']].mean()
TTFG1MON_Index = df.groupby(pd.PeriodIndex(df['Date TTFG1MON Index'], freq='M'))[['TTFG1MON Index']].mean()
RPI = df.groupby(pd.PeriodIndex(df['Date'], freq='M'))[['RPI']].mean()
CPI = df.groupby(pd.PeriodIndex(df['Date'], freq='M'))[['CPI']].mean()

GBP_IRS_2y = df.groupby(pd.PeriodIndex(df['Date_GBP_IRS_2y'], freq='M'))[['GBP_IRS_2y']].mean()
GBP_IRS_10y= df.groupby(pd.PeriodIndex(df['Date_GBP_IRS_10y'], freq='M'))[['GBP_IRS_10y']].mean()


# swap_snippet= df.groupby(pd.PeriodIndex(df['Date_GBP_IRS_10y'], freq='M'))[['GBP_IRS_10y']].mean()


In [19]:
GBP_IRS_2y.index = GBP_IRS_2y.index.to_timestamp()
GBP_IRS_2y

,GBP_IRS_2y
Date_GBP_IRS_2y,
2009-09-01,1.822400
2009-10-01,1.840091
2009-11-01,1.861338
2009-12-01,1.828735
2010-01-01,1.832186
...,...
2021-08-01,0.472602
2021-09-01,0.568192
2021-10-01,0.978010


In [20]:
swap_snippet = bp_result_df[(bp_result_df.index>='2009-01-01') & (bp_result_df.index <= '2009-08-01')]['BPSW2 TRPU Curncy']
swap_snippet


Date BPSW2 TRPU Curncy
2009-01-01    2.122545
2009-02-01    2.129905
2009-03-01    2.142045
2009-04-01    2.143250
2009-05-01    2.031900
2009-06-01    2.238286
2009-07-01    2.132435
2009-08-01    1.898762
Name: BPSW2 TRPU Curncy, dtype: float64

In [21]:
swap_snippet2 = bp_result_df[(bp_result_df.index>='2022-01-01')]['BPSW2 TRPU Curncy']
swap_snippet2

Date BPSW2 TRPU Curncy
2022-01-01    1.894165
2022-02-01    2.230094
2022-03-01    2.186532
2022-04-01    2.365979
2022-05-01    2.538121
2022-06-01    2.933563
2022-07-01    2.700779
2022-08-01    3.368889
2022-09-01    2.475340
2022-10-01    2.349517
2022-11-01    2.344014
2022-12-01    2.392340
Name: BPSW2 TRPU Curncy, dtype: float64

In [22]:
GBP_IRS_2y = pd.concat([swap_snippet, GBP_IRS_2y['GBP_IRS_2y']])


In [23]:
GBP_IRS_2y = pd.concat([GBP_IRS_2y, swap_snippet2])
GBP_IRS_2y 

2009-01-01    2.122545
2009-02-01    2.129905
2009-03-01    2.142045
2009-04-01    2.143250
2009-05-01    2.031900
                ...   
2022-08-01    3.368889
2022-09-01    2.475340
2022-10-01    2.349517
2022-11-01    2.344014
2022-12-01    2.392340
Length: 168, dtype: float64

In [24]:
GBP_IRS_2y= pd.DataFrame({'GBP_IRS_2y':GBP_IRS_2y})

In [25]:
GBP_IRS_2y

,GBP_IRS_2y
2009-01-01,2.122545
2009-02-01,2.129905
2009-03-01,2.142045
2009-04-01,2.143250
2009-05-01,2.031900
...,...
2022-08-01,3.368889
2022-09-01,2.475340
2022-10-01,2.349517
2022-11-01,2.344014


In [26]:
GBP_IRS_10y

,GBP_IRS_10y
Date_GBP_IRS_10y,
2009-01,3.757500
2009-02,3.798333
2009-03,3.872500
2009-04,3.935500
2009-05,3.917500
...,...
2021-08,0.886550
2021-09,1.003097
2021-10,1.144687


In [27]:
GBP_IRS_10y.index = GBP_IRS_10y.index.to_timestamp()


In [28]:
GBP_IRS_10y

,GBP_IRS_10y
Date_GBP_IRS_10y,
2009-01-01,3.757500
2009-02-01,3.798333
2009-03-01,3.872500
2009-04-01,3.935500
2009-05-01,3.917500
...,...
2021-08-01,0.886550
2021-09-01,1.003097
2021-10-01,1.144687


In [29]:
swap_snippet10 = bp_result_df[(bp_result_df.index>='2022-01-01')]['BPSW10 TRPU Curncy']
swap_snippet10

Date BPSW2 TRPU Curncy
2022-01-01    1.725929
2022-02-01    1.866820
2022-03-01    1.899921
2022-04-01    2.085100
2022-05-01    2.240542
2022-06-01    2.626558
2022-07-01    2.302258
2022-08-01    2.688500
2022-09-01    2.155940
2022-10-01    2.063400
2022-11-01    2.046500
2022-12-01    2.104540
Name: BPSW10 TRPU Curncy, dtype: float64

In [30]:
GBP_IRS_10y = pd.concat([GBP_IRS_10y['GBP_IRS_10y'], swap_snippet10])


In [31]:
GBP_IRS_10y = pd.DataFrame({'GBP_IRS_10y':GBP_IRS_10y})

In [32]:
df_list = [CO1_Comdty,CL1_Comdty,GBP_Curncy,GBPEUR_Curncy,UKX_Index,SPX_Index,SX5E_Index,TTFG1MON_Index,CPI,RPI]

In [33]:
result_df = df_list[0]
for i in range(len(df_list)-1):
    if i<len(df_list):
        result_df = result_df.join(df_list[i+1], how='left')

In [35]:
result_df.index = result_df.index.to_timestamp()

In [ ]:
corr_matrix = result_df.corr()

In [ ]:
sns.heatmap(corr_matrix, annot=True)

In [40]:
result_df = result_df.join(GBP_IRS_2y, how = 'left').join(GBP_IRS_10y, how = 'left')

In [41]:
result_df

,CO1 Comdty,CL1 Comdty,GBP Curncy,GBPEUR Curncy,UKX Index,SPX Index,SX5E Index,TTFG1MON Index,CPI,RPI,GBP_IRS_2y,GBP_IRS_10y
Date CO1 Comdty,,,,,,,,,,,,
1988-06-01,15.248333,16.054091,1.769541,NaN,1838.245455,270.356818,753.058182,NaN,49.7,106.6,NaN,NaN
1988-07-01,14.922381,15.635000,1.737955,NaN,1832.400000,268.579091,767.135909,NaN,49.7,106.7,NaN,NaN
1988-08-01,15.009091,15.696087,1.731052,NaN,1802.827273,262.703913,760.152609,NaN,49.9,107.9,NaN,NaN
1988-09-01,13.540909,14.973182,1.719150,NaN,1789.640909,268.413636,783.442727,NaN,50.1,108.4,NaN,NaN
1988-10-01,12.549048,14.744000,1.766640,NaN,1839.495000,274.702500,803.919000,NaN,50.3,109.5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-01,111.959545,106.196667,1.247484,1.178074,7472.767778,4084.583529,3745.749444,101.137500,120.8,337.1,2.538121,2.240542
2022-06-01,117.503182,108.356111,1.233611,1.167758,7238.188421,3913.432222,3572.135789,119.347368,121.8,340.0,2.933563,2.626558
2022-07-01,105.123810,99.030000,1.231479,1.184589,7303.865263,4062.266316,3652.041053,158.477368,122.5,343.2,2.700779,2.302258


In [ ]:
bp_result_df.index = bp_result_df.index.to_timestamp()

In [42]:
result_df.index[56]>=datetime.strptime('2008', '%Y')

False

In [ ]:
bp_result_df.index[56]>=datetime.strptime('2008', '%Y')

In [ ]:
result_df

In [43]:
result_df_post_housing = result_df[result_df.index >= datetime.strptime('2008', '%Y')]

In [44]:
result_df = result_df_post_housing

In [45]:
# make new DataFrame to have items in GBP currency:
result_GBP_df = result_df.copy()
# convert commodity prices and US equity prices from USD to GBP:
result_GBP_df['CO1 Comdty'] = result_df['CO1 Comdty']/result_df['GBP Curncy']
result_GBP_df['CL1 Comdty'] = result_df['CL1 Comdty']/result_df['GBP Curncy']
result_GBP_df['TTFG1MON Index'] = result_df['TTFG1MON Index']/result_df['GBP Curncy']
result_GBP_df['SPX Index'] = result_df['SPX Index']/result_df['GBP Curncy']
# convert Eurostoxx price from EUR to GBP:
result_GBP_df['SX5E Index'] = result_df['SX5E Index']/result_df['GBPEUR Curncy']

In [46]:
house_prices = pd.read_csv('Nationwide_House_Price_Index.csv')

In [47]:
house_prices_df = house_prices.rename({'Unnamed: 0': 'Date'}, axis=1)
house_prices_df['Date'] = pd.to_datetime(house_prices_df['Date'], infer_datetime_format=True)
house_prices_df = house_prices_df.set_index('Date')
house_prices_df

,UKNBALL Index
Date,
1991-01-01,105.8
1991-02-01,106.7
1991-03-01,105.5
1991-04-01,107.1
1991-05-01,108.5
...,...
2022-04-01,533.9
2022-05-01,538.4
2022-06-01,541.8


In [48]:
result_GBP_df = result_GBP_df.join(house_prices_df)

In [50]:
result_GBP_df

,CO1 Comdty,CL1 Comdty,GBP Curncy,GBPEUR Curncy,UKX Index,SPX Index,SX5E Index,TTFG1MON Index,CPI,RPI,GBP_IRS_2y,GBP_IRS_10y,UKNBALL Index
Date CO1 Comdty,,,,,,,,,,,,,
2008-01-01,47.499136,49.032901,1.935078,1.312152,5703.794545,679.783617,2809.434399,13.013333,82.4,209.8,NaN,NaN,360.0
2008-02-01,49.061810,52.576833,1.929500,1.301310,5799.242857,684.591604,2818.252310,12.690186,83.0,211.4,NaN,NaN,357.8
2008-03-01,53.245534,53.085652,1.931927,1.272118,5533.194500,661.717209,2738.417350,13.157845,83.4,212.1,NaN,NaN,357.3
2008-04-01,57.171016,55.750359,1.931565,1.259265,5798.256957,684.573642,2869.478405,13.528677,84.0,214.0,NaN,NaN,356.2
2008-05-01,64.980361,60.524628,1.918755,1.259209,5863.045000,696.741145,2873.532809,13.459903,84.6,215.1,NaN,NaN,346.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-01,89.748267,85.128666,1.247484,1.178074,7472.767778,3274.256696,3179.554466,81.073170,120.8,337.1,2.538121,2.240542,538.4
2022-06-01,95.251442,87.836565,1.233611,1.167758,7238.188421,3172.340166,3058.969505,96.746393,121.8,340.0,2.933563,2.626558,541.8
2022-07-01,85.363871,80.415504,1.231479,1.184589,7303.865263,3298.689210,3082.959231,128.688654,122.5,343.2,2.700779,2.302258,541.0


In [51]:
pct_change_list = []
for index, val in enumerate(result_GBP_df['RPI']):
    try:
        old = val
        new = result_GBP_df['RPI'][index+12]
        pct_change = ((new-old)/old)*100
        pct_change_list.append(pct_change)
    except:
        break
RPI_YOY = pd.DataFrame(pct_change_list)

In [52]:
RPI_YOY.index = result_GBP_df.index[12:]

In [53]:
final_df = result_GBP_df.join(RPI_YOY)[12:]

In [54]:
final_df = final_df.rename({0: 'RPI YOY'}, axis=1)


In [55]:
final_df = final_df.rename({'CO1 Comdty': 'Brent_Crude_GBP'}, axis=1)
final_df = final_df.rename({'CL1 Comdty': 'WTI_Crude_GBP'}, axis=1)
final_df = final_df.rename({'TTFG1MON Index': 'Natural_Gas'}, axis=1)
final_df = final_df.rename({'UKNBALL Index': 'Nationwide_HPI'}, axis=1)
final_df = final_df.rename({'Date CO1 Comdty': 'Date'}, axis=1)


In [56]:
final_df.index = final_df.index.rename('Date')

In [57]:
final_df

,Brent_Crude_GBP,WTI_Crude_GBP,GBP Curncy,GBPEUR Curncy,UKX Index,SPX Index,SX5E Index,Natural_Gas,CPI,RPI,GBP_IRS_2y,GBP_IRS_10y,Nationwide_HPI,RPI YOY
Date,,,,,,,,,,,,,,
2009-01-01,30.903838,33.424333,1.479027,1.099905,4308.390000,596.071497,2135.982324,12.472805,84.9,210.1,2.122545,3.757500,300.2,0.142993
2009-02-01,29.393166,33.617553,1.492643,1.120824,4232.114762,570.410394,2020.716905,9.467866,85.6,211.4,2.129905,3.798333,294.7,0.000000
2009-03-01,31.733352,36.826152,1.494241,1.097877,4107.840909,566.393975,2029.160819,8.064295,85.8,211.3,2.142045,3.872500,301.1,-0.377181
2009-04-01,33.990786,36.012519,1.511936,1.125327,4259.890500,588.042125,2119.383029,7.762397,86.0,211.5,2.143250,3.935500,302.9,-1.168224
2009-05-01,37.524515,38.801542,1.561252,1.131686,4434.719500,582.718727,2164.120107,7.757394,86.4,212.8,2.031900,3.917500,307.2,-1.069270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-01,89.748267,85.128666,1.247484,1.178074,7472.767778,3274.256696,3179.554466,81.073170,120.8,337.1,2.538121,2.240542,538.4,11.659490
2022-06-01,95.251442,87.836565,1.233611,1.167758,7238.188421,3172.340166,3058.969505,96.746393,121.8,340.0,2.933563,2.626558,541.8,11.842105
2022-07-01,85.363871,80.415504,1.231479,1.184589,7303.865263,3298.689210,3082.959231,128.688654,122.5,343.2,2.700779,2.302258,541.0,12.340426


In [59]:
final_df[final_df.isna()].sum()
#final_df.info()
#found that 10yr and 2yr swaps have data missing from 2011-2014

Brent_Crude_GBP    0.0
WTI_Crude_GBP      0.0
GBP Curncy         0.0
GBPEUR Curncy      0.0
UKX Index          0.0
SPX Index          0.0
SX5E Index         0.0
Natural_Gas        0.0
CPI                0.0
RPI                0.0
GBP_IRS_2y         0.0
GBP_IRS_10y        0.0
Nationwide_HPI     0.0
RPI YOY            0.0
dtype: float64

In [ ]:
pwd


In [60]:
final_df.to_csv('final_df.csv')